In [6]:
import sys
sys.path.append('../')
from mvcomp import *

import os
from os.path import join, basename

import glob

# Make lists of subjects 

In [4]:
# For one-sample analysis, only one list is needed

# For two-sample analysis (where the reference is a specific group, e.g., control): 
# make a list for the reference group and a list of subjects you want to compute D2 for


# Let's use the example case of one-sample analysis here (with the leave-one-subject-out approach)

# Directory containing the folders of each subject (e.g., '/project/data/Subject_001/')
in_dir = '/project/data/'

# P_folders is a list with the full path of each subject
P_folders = glob.glob(join(in_dir,'*'))

ID_list = []

for i in P_folders: 
    # PID is the basename of every element in P_folders. ID_list will be a list containing all PIDs
    PID = basename(i)
    ID_list.append(PID)
    
print(len(ID_list))

0


# Computing the reference mean and covariance matrix

In [8]:
# Compute average of all subjects (for each feature) using compute_average function
# Since we're using the leave-one-subject-out approach, we will only use the average maps for getting the correlation matrix (to visualize relationships between metrics)
# Skip this cell if you are not interested in visualizing the correlation matrix

feature_dir = '/project/data/'
out_dir = '/project/data/Group_average/'
features = ['CSD_AFDtotal', 'CSD_meanFC', 'CSD_sumFDC', 'DTI_AD', 'DTI_FA', 'DTI_MD', 'DTI_RD', 'MPM_MTsat', 'MPM_PD', 'MPM_R1', 'MPM_R2', 'NODDI_ICVF', 'NODDI_ISOVF', 'NODDI_OD', 'T1_div_by_T2']
feature_suffix = '_warped_group_space.nii.gz' # features and feature_suffix should create the file names for the features wanted

# We provided the compute_average function to create custom averages when desired
compute_average(ids= ID_list, in_dir= feature_dir, out_dir, features,
                        feature_suffix, verbose=1)


SyntaxError: positional argument follows keyword argument (<ipython-input-8-381f32613b9a>, line 11)

In [ ]:
# Create a list of feature names and a list of paths for the reference images (this list will be used for computation of correlation matrix)

reference_dir = '/project/data/Group_average/'
reference_suffix = '_xxaverage.nii.gz' # edit this with the suffix of the images outputted in previous step (the suffix depends on the number of subjects contained in the average)
excluded_features = []  # add names of features you want to exclude (e.g., 'T1_div_by_T2') 

# get reference path and file names
reference_fullpath_list, reference_fname_list = feature_list(reference_dir, 
                                                    reference_suffix, excluded_features)

print(reference_fname_list)
print(reference_fullpath_list)

# Create feature matrix of the reference within a mask (and using a threshold) 

# Define mask
mask_f = '/project/data/mask/white_matter_mask.nii.gz'

# generate matrices of the reference features and the masks, with a desired threshold
m_f_mat, mask_img, mat_mask = feature_gen(reference_fullpath_list, mask_image_fname = mask_f, 
                                            mask_threshold = 0.9)

print("reference feature matrix shape is: ", m_f_mat.shape)
# m_f_mat is the feature matrix (from the reference)
# mask_img is a nibabel object of the mask
# mat_mask is a boolian numpy array of the mask with the value of zero in the inf\nan voxels 


In [ ]:
# You can plot the covariance and correlation of your features should you want to inspect them 
# Calculate covariance (s) and inverse of covariance (pinv_s) of the reference's feature matrix
s, pinv_s = norm_covar_inv(m_f_mat, mat_mask)

# Show correlation coefficient map 
correlation_fig(s, reference_fname_list)

# Computing D2

In [ ]:
# Compute D2 (with leave-one-out approach; exclude_comp_from_mean_cov= True)
# At the moment, model_dir and suffix_name_model are needed just to get features names (this has to be fixed)

result_dict = model_comp(feature_in_dir= feature_dir, model_dir= None, 
                                 suffix_name_comp= feature_suffix, 
                                 exclude_comp_from_mean_cov= True, suffix_name_model= reference_suffix, 
                                 mask_f= mask_f, mask_img= None, verbosity=1, 
                                 mask_threshold=0.99, subject_ids= ID_list, 
                                 exclude_subject_ids= None, feat_sub= excluded_features, 
                                 return_raw=False)

# results are stored in a dictionary, can access array of D2 data (of all subjects) like this:
all_D2 = result_dict['all_dist']
print(all_D2.shape) # Shape will be (number of voxels) x (number of subjects)

# Save matrix containing all D2 data as a numpy array

np.save('/project/Analyses/Reference_all_subjects/D2_all_subjects.npy', all_D2)


In [ ]:
# To save the D2 maps (images)
# Will also plot the mean D2 map and D2 histogram of all subjects 
# Right now mask_img is needed (which is created with feature_gen function)

subject_ids = ID_list

dist_plot(all_D2, all_masks, subject_ids, excluded_features, save_results=True, 
              out_dir = '/project/Analyses/Reference_all_subjects/', 
              mask_f=None, mask_img=mask_img, coordinate=(-10, -50, 10), vmin = 0, vmax = 70, hist_tr = 120, nobin = 200)

# Change name of directory created if needed

## Determining feature importance 

In [ ]:
# Extracting distances per feature (raw) using the return_raw=True option of model_comp
# Note that this step can be combined with the previous one, but it will increase computational time.

result_dict = model_comp(feature_in_dir= feature_dir, model_dir= None, 
                                 suffix_name_comp= feature_suffix, 
                                 exclude_comp_from_mean_cov= True, suffix_name_model= reference_suffix, 
                                 mask_f= mask_f, mask_img= None, verbosity=1, 
                                 mask_threshold=0.99, subject_ids= ID_list, 
                                 exclude_subject_ids= None, feat_sub= excluded_features, 
                                 return_raw=True)


raw_D2 = result_dict['raw_dist'] 
print(raw_D2.shape) # Shape will be (number of voxels) x (number of features) x (number of subjects)

# The raw distances can then be summarized to obtain the relative contribution of each feature

In [ ]:
def correlation_fig_big(s, f_list):
    '''
    Plot the correlation table from the covariance matrix.
    
    Args: 
        s (numpy.ndarray): covariance matrix of size feature x feature
        f_list (List of strings): A list of the names of the features that should be in the same order of the covarinca matrix.
        
    Return:
        Non.
    
    
    '''

    r = np.zeros(s.shape)
    for i in range(s.shape[0]):
        for j in range(s.shape[1]):
            r[i, j] = s[i, j] / (math.sqrt(s[i, i] * s[j, j]))

    fig, ax = plt.subplots(figsize=(11.2, 8.4))
    im = ax.imshow(r, cmap='RdBu_r', vmin=-1, vmax=1)
    title = ax.set_title('Metric-Metric Correlation Matrix')
    cbar = ax.figure.colorbar(im, ax=ax)
    ax.set_xticks(np.arange(s.shape[1]))
    ax.set_yticks(np.arange(s.shape[0]))
    xtl = ax.set_xticklabels(f_list)
    ytl = ax.set_yticklabels(f_list)  # Let the horizontal axes labeling appear on top.
    _ = ax.tick_params(top=True, bottom=False,
                       labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
    _ = plt.setp(ax.get_xticklabels(), rotation=-30, ha="right",
                 rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    for i in range(len(f_list)):
        for j in range(len(f_list)):
            if i == j:
                pass
            else:
                val = r[i, j]
                if val > 0.6:
                    cc = 'w'
                else:
                    cc = 'k'
                text = ax.text(j, i, "{:.2f}".format(val),
                               ha="center", va="center", color=cc)